# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [128]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf
import pandas_ta
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'statsmodels'

## Importing S&P 500 from Wikipedia

In [9]:
# stocks = pd.read_csv('sp_500_stocks.csv')
# stocks

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-12-15'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed


adj close       close        high         low        open  \
date       ticker                                                               
2015-12-17 A        38.179028   40.779999   41.630001   40.770000   41.549999   
           AAL      40.576706   42.459999   43.950001   42.450001   43.330002   
           AAPL     24.803656   27.245001   28.062500   27.245001   28.004999   
           ABBV     39.907444   56.380001   57.930000   56.349998   57.660000   
           ABT      38.962093   45.259998   46.220001   45.250000   46.090000   
...                       ...         ...         ...         ...         ...   
2023-12-14 YUM     131.110001  131.110001  131.910004  129.350006  131.619995   
           ZBH     118.529999  118.529999  120.330002  118.430000  119.089996   
           ZBRA    272.160004  272.160004  274.440002  247.110001  248.559998   
           ZION     45.669998   45.669998   46.150002   43.740002   43.820000   
           ZTS     200.089996  200.089996  201.919998  198.000000  198.000000   

                        volume  
date       ticker               
2015-12-17 A         2218800.0  
           AAL       7225000.0  
           AAPL    179091200.0  
           ABBV      7848900.0  
           ABT       5792200.0  
...                        ...  
2023-12-14 YUM       3238700.0  
           ZBH       2049000.0  
           ZBRA      1165900.0  
           ZION      6563800.0  
           ZTS       3044400.0  

[994308 rows x 6 columns]

## Making Our First API Call

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [22]:
symbol = 'AAPL'

aapl = yf.Ticker(symbol)
aapl_info = aapl.info
mc = aapl_info['marketCap']
price = aapl_info['currentPrice']

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [23]:
cols = ['Ticker', 'Stock Price', 'Market Cap', 'Number of Shares to Buy']
res_df = pd.DataFrame(columns = cols)
res_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy


In [57]:
# pd.join(res_df, pd.Series([
#         symbol,
#         price,
#         mc,
#         'N/A'
#     ],
#     index = cols    
#     )
# )
# res_df.loc[len(res_df)] = [symbol, price, mc, 'N/A']

res_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,AAPL,194.83,3030152118272,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [63]:
res_df = pd.DataFrame(columns = cols)

symbols_string = " ".join(str(element) for element in symbols_list)
tickers = yf.Tickers(symbols_string)

symbols_string

'MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BK BBWI BAX BDX BRK-B BBY BIO TECH BIIB BLK BX BA BKNG BWA BXP BSX BMY AVGO BR BRO BF-B BLDR BG CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE COR CNC CNP CDAY CF CHRW CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FLT FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GEHC GEN GNRC GD GE GIS GM GPC GILD GPN GL GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUB

In [64]:
for symbol in symbols_list:
    info = tickers.tickers[symbol].info
    mc = info['marketCap']
    price = info['currentPrice']
    res_df.loc[len(res_df)] = [symbol, price, mc, 'N/A']

res_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,MMM,103.67,57258700800,N/A
1,AOS,80.23,11931805696,N/A
2,ABT,107.40,186452852736,N/A
3,ABBV,151.69,267814764544,N/A
4,ACN,339.50,212863778816,N/A
...,...,...,...,...
498,YUM,129.04,36170940416,N/A
499,ZBRA,261.61,13436184576,N/A
500,ZBH,120.00,25077719040,N/A
501,ZION,42.54,6302258176,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [73]:
val = input('Enter portfolio size')

try:
    portfolio_size = float(val)
except ValueError:
    print('Enter an Integer')

Enter portfolio size 100000


In [82]:
portfolio_size = 1000000
position_size = portfolio_size/len(res_df.index)

for i in range(0, len(symbols_list)):
    res_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/res_df.loc[i, 'Stock Price'])

res_df

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,MMM,103.67,57258700800,19
1,AOS,80.23,11931805696,24
2,ABT,107.40,186452852736,18
3,ABBV,151.69,267814764544,13
4,ACN,339.50,212863778816,5
...,...,...,...,...
498,YUM,129.04,36170940416,15
499,ZBRA,261.61,13436184576,7
500,ZBH,120.00,25077719040,16
501,ZION,42.54,6302258176,46


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [124]:
writer = pd.ExcelWriter('trades.xlsx', engine = 'xlsxwriter')
res_df.to_excel(writer, 'Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [125]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [126]:
# writer.sheets['Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Trades'].set_column('D:D', 18, string_format)
# writer.close()

writer.sheets['Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Trades'].write('C1', 'Market Cap', dollar_format)
writer.sheets['Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [127]:
col_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for col in col_formats.keys():
    writer.sheets['Trades'].set_column(f'{col}:{col}', 18, col_formats[col][1])
    writer.sheets['Trades'].write(f'{col}1', col_formats[col][0], string_format)


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()